# Importando as bibliotecas

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sn
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados
#from sklearn.metrics import plot_confusion_matrix
import time

In [116]:
# Dataset

In [23]:
procData = ProcessarDados("../dataset/bin_norm_10_features_m73_n4.csv", sep = ',')

In [24]:
# Funções

In [25]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_specificity = [] # taxa de verdadeiros negativos ou especificidade
    resultados_f2 = []
    resultados_parametros = []
    resultados_matriz = []
    resultados_time_train = []
    resultados_time_test = []
    resultados_time_total = []

    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        seed = i
        X_train, X_test, y_train, y_test = procData.holdout2(0.2, seed)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor K-Vizinhos e pesos, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = KNeighborsClassifier()
        grid_knn = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        
        start_time_train = time.clock()
        grid_knn.fit(X_train, y_train)
        
        # Treinando do modelo com os melhores parametros encontrados
        grid_knn.best_params_
        Kvizinhos_best = int(grid_knn.best_params_['n_neighbors']) 
        p_best = grid_knn.best_estimator_.p
        KNN = KNeighborsClassifier(n_neighbors=Kvizinhos_best, p=p_best, metric='minkowski')
        KNN.fit(X_train, y_train)
        
        time_train = time.clock() - start_time_train

        #testando o modelo
        start_time_test = time.clock()
        y_pred = KNN.predict(X_test)
        time_test = time.clock() - start_time_test
        cm  = confusion_matrix(y_test, y_pred)
        if exibir_matriz_confusao:
            print(cm)

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        # f2-score
        # Fbeta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
        beta = 0.5
        f2_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn+fp)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_specificity.append(specificity)
        resultados_f2.append(f2_score)

        best_parametros = {'n_neighbors': Kvizinhos_best, 'p': p_best};
        resultados_parametros.append(best_parametros)

        resultados_time_train.append(time_train)
        resultados_time_test.append(time_test) 
        resultados_time_total.append(time_train+time_test)

        if exibir_metricas:
            print("Rodada:",i)
            print("Parâmetros:",best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("Specificity:",specificity)
            print("fb-Score:",f2_score)
            print("Time Train (s):",time_train)
            print("Time Test (s):",time_test)
            print("Time Total (s):",time_train+time_test)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['specificity'] = resultados_specificity
    resultados_gerais['f2'] = resultados_f2
    resultados_gerais['params'] = resultados_parametros
    resultados_gerais['time_train'] = resultados_time_train
    resultados_gerais['time_test'] = resultados_time_test
    resultados_gerais['time_total'] = resultados_time_total
    
    return resultados_gerais

In [26]:
import csv
def writeResultsCsv(dict_results, name_file):
    keys = list(dict_results.keys())
    keys.insert(0,'Rodada')
    with open(name_file, 'w') as f:  # You will need 'wb' mode in Python 2.x
        w = csv.writer(f)
        #w.writeheader()
        row = []
        w.writerow(keys)
        for i in range(epocas):
            row = []
            acc = dict_results.get('accuracy')[i]
            prec = dict_results.get('precision')[i]
            sen = dict_results.get('recall')[i]
            spe = dict_results.get('specificity')[i]
            fscore = dict_results.get('f2')[i]
            param = dict_results.get('params')[i]
            time_train = dict_results.get('time_train')[i]
            time_test = dict_results.get('time_test')[i]
            time_total = dict_results.get('time_total')[i]

            row.append(i+1)
            row.append(acc)
            row.append(prec)
            row.append(sen)
            row.append(spe)
            row.append(fscore)
            row.append(param)
            row.append(time_train)
            row.append(time_test)
            row.append(time_total)

            w.writerow(row)
    

In [27]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== RESULTADOS DAS MÉTRICAS DO MODELO ==============================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\tSpecificity \t|\t fb-Score")
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end=' ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='  ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end=' ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['specificity'], axis=0), np.std(dict_metricas['specificity'], axis=0)),end='   ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['f2'], axis=0), np.std(dict_metricas['f2'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)
        writeResultsCsv(dict_metricas, 'resultados_knn.csv')

# Definindo os parâmetros

In [28]:
#K-Vizinhos
n_neighbors = np.arange(1,25)
#'uniforme': Todos os pontos em cada vizinho têm o mesmo peso.
#'distância': Inverso de sua distância, os vizinhos mais próximos de um ponto de consulta terão uma influência maior do que os vizinhos mais distantes.
pesos = [1,2]
param_grid = {'n_neighbors': n_neighbors, 'p': pesos}
epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True

# Treinando e obtendo as métricas do modelo

In [29]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('KNN',dict_metricas)

[[15  5]
 [ 3 47]]
Rodada: 0
Parâmetros: {'n_neighbors': 16, 'p': 2}
Accuracy: 0.8857142857142857
Precision: 0.9038461538461539
Recall: 0.94
Specificity: 0.75
fb-Score: 0.9108527131782944
Time Train (s): 1.6556483000003936
Time Test (s): 0.004339200000686105
Time Total (s): 1.6599875000010798


[[18  2]
 [ 2 48]]
Rodada: 1
Parâmetros: {'n_neighbors': 16, 'p': 2}
Accuracy: 0.9428571428571428
Precision: 0.96
Recall: 0.96
Specificity: 0.9
fb-Score: 0.96
Time Train (s): 1.4835448000012548
Time Test (s): 0.004228399999192334
Time Total (s): 1.487773200000447


[[17  3]
 [ 0 50]]
Rodada: 2
Parâmetros: {'n_neighbors': 11, 'p': 2}
Accuracy: 0.9571428571428572
Precision: 0.9433962264150944
Recall: 1.0
Specificity: 0.85
fb-Score: 0.9541984732824428
Time Train (s): 1.4183622000000469
Time Test (s): 0.004130799999984447
Time Total (s): 1.4224930000000313


[[18  2]
 [ 3 47]]
Rodada: 3
Parâmetros: {'n_neighbors': 14, 'p': 2}
Accuracy: 0.9285714285714286
Precision: 0.9591836734693877
Recall: 0.94
Sp